<a href="https://colab.research.google.com/github/Leonnello/sjf-memory-placement-sim/blob/leon-patch1/ITS150L_SA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ITS150L_FOPI01_1Q2425<br>
Members:<br>
Celina Therese Tabano<br>
Noel Allen Elises

In [ ]:
#### Imports
import random
import os
import signal
import pickle

In [ ]:
class Process:
  position = None

  def __init__(self, pid, burst_time, memory_size):
    self.pid = pid
    self.burst_time = burst_time
    self.memory_size = memory_size

  # decrease remaining time
  def run(self):
    self.burst_time -= 1

    if self.burst_time == 0:
      self.position.isFree = True
      self.position = None


In [ ]:
class MemoryBlock:
  def __init__(self, start, end, isFree: bool = True):
    self.start = start
    self.end = end
    self.isFree = isFree

  def get_size(self):
    return self.end - self.start + 1

In [ ]:
##### GLOBAL VARIABLES
# counter for pid
pid_count = 1

# array of all process to be scheduled
current_processes = []

# array for all memory blocks
memory_blocks = []

time = 0

In [ ]:
def generate_blocks(size : int):
  # init rng
  random.seed()

  # select random num of blocks
  n_blocks = 0
  if size <= 500:
    n_blocks = random.randrange(3, 10)
  else:
    n_blocks = random.randrange(4, round(size/100))

  end_pos = random.sample(range(1,size), n_blocks)
  end_pos.sort()

  print(end_pos)

  # divide and create blocks
  for i in range(len(end_pos)):
    if i == 0:
      memory_blocks.append(MemoryBlock(0, end_pos[i]))
    elif i == len(end_pos) - 1:
      memory_blocks.append(MemoryBlock(memory_blocks[i-1].end + 1, size - 1))
    else:
      memory_blocks.append(MemoryBlock(memory_blocks[i-1].end + 1, end_pos[i]))


In [ ]:
# function to input new jobs to queue
# prolly on a different process/fork?
def new_job():
  global pid_count
  while True:
    try:
      burst_time = int(input(f"Enter process#{pid_count} burst time: "))
      memory_size = int(input(f"Enter process#{pid_count} memory size: "))
      current_processes.append(Process(pid_count,burst_time, memory_size))
      pid_count += 1
      break
    except ValueError:
      print("Invalid input. Enter whole numbers only.")

In [ ]:
def pass_time(processes):
  global time

  # pass time for each process that is already fitted in memory block
  for n in range(len(processes)):
    if processes[n].position != None:
      time += 1
      processes[n].run()

    if processes[n].burst_time <= 0:
      processes[n] = None


In [ ]:
# function to send current_processes for sjf scheduling
# run this whenever all the processes in a sjf batch is complete?
def sjf_scheduling(processes):
  global time
  # clear array for next batch
  global current_processes
  current_processes.clear()

  # sort processes based on Burst Time
  processes = sorted(processes, key=lambda process: process.burst_time)

  # get total wt and tat
  # TO REMOVE LATER
  wt,  tat = [], []
  total_wt, total_tat = 0, 0

  for i in range(len(processes)):
    if processes[i] == None:
      continue
    print(f"PID {processes[i].pid}: {processes[i].burst_time}ms {processes[i].memory_size}Kb")

    #calcs moved here - gets affected by burst decrement
    if (i == 0):
      wt.append(0)
      tat.append( processes[i].burst_time)
    else:
      wt.append( processes[i-1].burst_time + wt[i-1] )
      tat.append( processes[i].burst_time + wt[i] )

    total_wt += wt[i]
    total_tat += tat[i]

  for i in range(len(processes)):
    if processes[i] == None:
      continue

    while not best_fit(processes[i]):
    # if can't fit, pass time till there is available hole
      pass_time(processes)
      # compact()
      coalesce()

    # pass time every successful fit
    pass_time(processes)

    # REMOVE Later, dupp
    if processes[i] == None:
      continue

    #repaint()

    #compact()

    #coalesce()

  print("Average WT: ", total_wt/len(processes))
  print("Average TAT: ", total_tat/len(processes))

In [ ]:
def best_fit(job_to_fit) -> bool:
  global memory_blocks
  difference = None
  block_index = -1

  for i in range(len(memory_blocks)):
    if memory_blocks[i].isFree and memory_blocks[i].get_size() >= job_to_fit.memory_size:
      current_diff = memory_blocks[i].get_size() - job_to_fit.memory_size
      if difference == None or current_diff < difference:
        difference = current_diff
        block_index = i

  # if fit is found
  if block_index != -1:
    # place job in the memory_block
    selected_block = memory_blocks[block_index]
    job_to_fit.position = selected_block
    selected_block.isFree = False

    end_pos = selected_block.end
    selected_block.end = selected_block.start + job_to_fit.memory_size - 1

    # create block for the remaining hole
    if difference != None and difference > 0:
      memory_blocks.append(MemoryBlock(selected_block.end + 1, end_pos))

    return True

  return False


In [ ]:
def is_in_array(array, element, *args) -> bool:
  for n in range(len(array)):
    match args:
      case ("start","end"):
        if array[n].start == element.start and array[n].end == element.end:
          return True

  return False

In [ ]:
# coalesce
def coalesce():
  global memory_blocks
  coalesced_blocks = []

  # sort by start #
  memory_blocks = sorted(memory_blocks, key=lambda block: block.start)

  if len(memory_blocks) == 1:
    coalesced_blocks = memory_blocks
  else:
    prev = None
    for i in range(len(memory_blocks) - 1):
      current = memory_blocks[i]
      next = memory_blocks[i+1]

      if prev is not None:
        # if previous was merged, update current
        if prev.isFree and prev.end == current.end:
          current = prev

      if current.isFree and next.isFree:
        # if prev is current, meaning prev and current will be merged
        # delete duplicate from final array
        if prev is current:
          coalesced_blocks.pop()

        current.end = next.end

      if not is_in_array(coalesced_blocks, current, "start", "end"):
        coalesced_blocks.append(current)

      if i == len(memory_blocks) - 2:
        if not is_in_array(coalesced_blocks, next, "start", "end"):
          coalesced_blocks.append(next)

      prev = current

  memory_blocks = coalesced_blocks.copy()


In [ ]:
def compact():
  global memory_blocks

  #ignore function based on time interval
  if time == 0 or time % 20 != 0: return

  memory_blocks = sorted(memory_blocks, key=lambda block: block.start)

  #count non-free blocks
  process_count = sum(1 for block in memory_blocks if not block.isFree)

  #get all non-free blocks (biggest to smallest) then clear
  process_blocks = sorted([block for block in memory_blocks if not block.isFree], key=lambda block: block.get_size(), reverse = True)
  memory_blocks.clear()

  end = -1 # for first loop

  #change positions then append
  for block in process_blocks:
    start = end + 1
    end = block.get_size() - 1
    memory_blocks.append(MemoryBlock(start, end, False))

In [ ]:
def main():
  global current_processes, pid_count
  global memory_blocks
  global time

  main_memory_size = 0
  # pipe for interprocess comms
  pipe_r, pipe_w = os.pipe()

  # container to serialize/deserialize data for pipe
  data_container = {"processes" : current_processes}

  while True:
    try:
      main_memory_size = int(input("Enter memory size of the program: "))
      break
    except ValueError:
      print("Invalid input. Enter whole numbers only.")

  generate_blocks(main_memory_size)

  pid = os.fork()

  if pid == 0:
    # child process for the job simulation and execution

    # close write
    os.close(pipe_w)

    # child main process loop
    while True:
      read = os.read(pipe_r, 4096)
      if read:
        # deserialize data from pipe
        data = pickle.loads(read)

        # update current_processes in child process
        current_processes = data["processes"]

      # maybe start sjf_schedule when there's at least 5 process?
      # or just run it whenever it can ig
      if len(current_processes) >= 5:
        sjf_scheduling(current_processes.copy())

        # test display
        print(f"Time: {time}ms")
        memory_blocks = sorted(memory_blocks, key=lambda block: block.start)
        for i in range(len(memory_blocks)):
          print(f"[{memory_blocks[i].start}, {memory_blocks[i].end}] {memory_blocks[i].isFree}")

      # compact()
      # coalesce()
      #placeProcess()
    os.close(pipe_r)
    print("Child exited.")

  elif pid > 0:
    # parent process for inputting new jobs real time?

    # close read
    os.close(pipe_r)

    while True:
      new_job()

      # serialize and pass updated data to child process
      serialized_data = pickle.dumps(data_container)
      os.write(pipe_w, serialized_data)

      user_prompt = input("Enter any to continue or 'exit' to quit: ")
      if user_prompt.lower() == 'exit':
        #send exit signal to child then wait
        os.close(pipe_w)

        os.kill(pid, signal.SIGTERM)
        os.waitpid(pid, 0)
        print("Program exited successfully")
        break

main()

Enter memory size of the program: 1000
[69, 164, 210, 254, 416]
Enter process#1 burst time: 25
Enter process#1 memory size: 25
Enter any to continue or 'exit' to quit: 25
Enter process#2 burst time: 25
Enter process#2 memory size: 25
Enter any to continue or 'exit' to quit: 25
Enter process#3 burst time: 25
Enter process#3 memory size: 25
Enter any to continue or 'exit' to quit: 25
Enter process#4 burst time: 25
Enter process#4 memory size: 25
Enter any to continue or 'exit' to quit: 25
Enter process#5 burst time: 25
Enter process#5 memory size: 25
PID 1: 25ms 25Kb
PID 2: 25ms 25Kb
PID 3: 25ms 25Kb
PID 4: 25ms 25Kb
PID 5: 25ms 25Kb
Average WT:  50.0
Average TAT:  75.0
Time: 5s
[0, 24] False
[25, 49] False
[50, 69] True
[70, 94] False
[190, 210] True

[95, 164] True[165, 189] False
[211, 235] False
[236, 254] True
[255, 999] True
Enter any to continue or 'exit' to quit: exit
Program exited successfully


--------------------------------------------------------------------------------

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output
import time

In [ ]:
#part of static() - repaints static visuals
def repaintStatic(drawObj):
  # Set font (optional, use default if not available)
  try:
      font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 24)
  except IOError:
      font = ImageFont.load_default()

  # Draw the rectangle (outline only, no fill)
  drawObj.rectangle([(50, 50), (1000, 250)], outline="black", width=5)

  # Add text/label inside the rectangle
  text = "Memory"
  text_position = (50, 35)  # Position where the text will be placed

  # Add text to the image
  drawObj.text(text_position, text, fill="black", font=font)

In [ ]:
memory_blocks.clear()
memory_blocks.append(MemoryBlock(0,99))
memory_blocks.append(MemoryBlock(100,149))
memory_blocks.append(MemoryBlock(150,199))
memory_blocks.append(MemoryBlock(200,249, False))
memory_blocks.append(MemoryBlock(250,349))
memory_blocks.append(MemoryBlock(350,499, False))


In [ ]:
#updates image with the changes in memory_blocks array
# canvas init
image = Image.new("RGB", (1000, 300), color="white")
#create object that contains the canvas
drawObj = ImageDraw.Draw(image)
def repaint():
  global memory_blocks
  currentPos = (50, 50)



  repaintStatic(drawObj)
  #display(image)

  for block in memory_blocks:
    # print(currentPos)
    # print(p2)
    # print()
    p2 = (currentPos[0] + block.get_size(), 250)

    drawObj.rectangle([currentPos, p2], outline="blue", width=2)

    #increment pointer with block size
    currentPos = (currentPos[0] + block.get_size(), currentPos[1])

    clear_output(wait=True)
    display(image)
    #frame rate
    time.sleep(0.5)

  print("repaint complete.")

In [ ]:
repaint()